In [19]:
import openpyxl
from openpyxl import load_workbook
import numpy as np
import pandas as pd

In [20]:
pathExcel = r"G:\Drives compartilhados\Pontte Crédito\0_HOME EQUITY\0_Analises\MARIA SONIA PAULO DA SILVA ID 586505100\KIT QI\Cálculo_Fluxo.xlsx"

In [21]:
wb = load_workbook(filename=pathExcel, 
                   read_only=True)
try:
    ws = wb['PRICE_CORR']
except:
    ws = wb['SAC_CORR']

# Read the cell values into a list of lists
data_rows = []
for row in ws['A18':'A500']:
    data_cols = []
    for cell in row:
        data_cols.append(cell.value)
    data_rows.append(data_cols)


In [22]:

# Transform into dataframe
df = pd.DataFrame(data_rows)
df.replace('',np.nan, inplace=True)
df = df.dropna()
lastRow = len(df) + 17
df.columns = ['numParcela']
try:
    df['numParcela'] = df['numParcela'].replace("I", 0)
except:
    print('erro')
df['numParcela'] = df['numParcela'].astype(int)

In [23]:
#Append Data Vencimento
date_rows = []
for row in ws['B18':f'B{lastRow}']:
    date_cols = []
    for cell in row:
        date_cols.append(cell.value)
    date_rows.append(date_cols)

df["data_vencto"] = pd.DataFrame(date_rows)

In [24]:
#Append Valor Amortização
date_rows = []
for row in ws['C18':f'C{lastRow}']:
    date_cols = []
    for cell in row:
        date_cols.append(cell.value)
    date_rows.append(date_cols)
df["amortizacao"] = pd.DataFrame(date_rows)

In [25]:
#Append Juros
date_rows = []
for row in ws['D18':f'D{lastRow}']:
    date_cols = []
    for cell in row:
        date_cols.append(cell.value)
    date_rows.append(date_cols)
df["juros"] = pd.DataFrame(date_rows)

In [26]:
numCCI = "123456"

In [27]:
df['amortizacao'] = df['amortizacao'].replace(np.nan, 0)
df['juros'] = df['juros'].replace(np.nan, 0)
df['valor_principal'] = round(df['amortizacao'],2)
df['valor_futuro'] = round(df['amortizacao']+df['juros'],2)

#Valor CCI
df['codigo_integracao'] = numCCI

#Valores Padrões
df['id_tipo_parcela'] = "2"
df['Aplica_Correcao'] = "S"
df['Aplica_Juros'] = "S"
df['periodicidade'] = "1"

In [28]:
df = df[['codigo_integracao', 'data_vencto', 'id_tipo_parcela', 'numParcela', 'valor_principal', 'valor_futuro', 'Aplica_Correcao', 'Aplica_Juros', 'periodicidade']]
df['data_vencto'] = pd.to_datetime(df['data_vencto'], format='%d/%m/%y').dt.strftime('%d/%m/%Y')

In [29]:
df_remove = df.loc[(df['valor_principal'] == 0)]
df = df.drop(df_remove.index)

In [30]:
if df['numParcela'].iloc[0] == 2:
    df['numParcela'] = df['numParcela'] - 1

df['numParcela'] = np.where(df['valor_principal'] >0, df['numParcela'].apply(str)+"/"+str(df['numParcela'].iloc[-1]), 0)
df = df.rename(columns={'numParcela': 'numero_parcela'})
df

,codigo_integracao,data_vencto,id_tipo_parcela,numero_parcela,valor_principal,valor_futuro,Aplica_Correcao,Aplica_Juros,periodicidade
2,123456,10/02/2023,2,1/120,2657.60,9920.23,S,S,1
3,123456,10/03/2023,2,2/120,2686.93,9920.23,S,S,1
4,123456,10/04/2023,2,3/120,2716.59,9920.23,S,S,1
5,123456,10/05/2023,2,4/120,2746.57,9920.23,S,S,1
6,123456,10/06/2023,2,5/120,2776.88,9920.23,S,S,1
...,...,...,...,...,...,...,...,...,...
117,123456,10/09/2032,2,116/120,9390.46,9920.23,S,S,1
118,123456,10/10/2032,2,117/120,9494.10,9920.23,S,S,1
119,123456,10/11/2032,2,118/120,9598.89,9920.23,S,S,1
120,123456,10/12/2032,2,119/120,9704.83,9920.23,S,S,1


In [16]:
import pprint
jsonTeste = df.to_json(orient='records').replace("\\/", str('/'))
pprint.pprint(jsonTeste)

'[{"codigo_integracao":"123456","data_vencto":"10/02/2023","id_tipo_parcela":"2","numero_parcela":"1/120","valor_principal":2657.6,"valor_futuro":9920.23,"Aplica_Correcao":"S","Aplica_Juros":"S","periodicidade":"1"},{"codigo_integracao":"123456","data_vencto":"10/03/2023","id_tipo_parcela":"2","numero_parcela":"2/120","valor_principal":2686.93,"valor_futuro":9920.23,"Aplica_Correcao":"S","Aplica_Juros":"S","periodicidade":"1"},{"codigo_integracao":"123456","data_vencto":"10/04/2023","id_tipo_parcela":"2","numero_parcela":"3/120","valor_principal":2716.59,"valor_futuro":9920.23,"Aplica_Correcao":"S","Aplica_Juros":"S","periodicidade":"1"},{"codigo_integracao":"123456","data_vencto":"10/05/2023","id_tipo_parcela":"2","numero_parcela":"4/120","valor_principal":2746.57,"valor_futuro":9920.23,"Aplica_Correcao":"S","Aplica_Juros":"S","periodicidade":"1"},{"codigo_integracao":"123456","data_vencto":"10/06/2023","id_tipo_parcela":"2","numero_parcela":"5/120","valor_principal":2776.88,"valor_fu

In [14]:
#Valores Cálculo Fluxo
dict_values = {'valorBrutoCF':"", 'tabelaCF': '', 'IOFCF':'', 'TACCF': '', 'valorRegistroCF': '', 'valorLiquidoCF': '', 'prazoMesesCF': '', 'taxaMesCF': '', 'primeiraParcelaCF':'',
                'valorImovelCF': '', 'carenciaCF': '', 'mesPularParcelaCF':'', 'CETCF':'', 'prazoContratoCF': ''
                }

#Transformar KEYS em lista
listKey =  list(dict_values.keys())
i = 0 

for row in ws['C2':'C15']:
    for cell in row:
        #Definição de valor da chave
        keyValue = listKey[i]
        i+=1

        valorCelula = cell.value

        if type(valorCelula) == float:
            valorCelula = round(valorCelula, 2)

        dict_values[keyValue] = valorCelula



In [15]:
dict_values

{'valorBrutoCF': 651751.35,
 'tabelaCF': 'Price',
 'IOFCF': 21597.03,
 'TACCF': 24000,
 'valorRegistroCF': 6154.32,
 'valorLiquidoCF': 600000,
 'prazoMesesCF': 120,
 'taxaMesCF': 0.01,
 'primeiraParcelaCF': 10184.28,
 'valorImovelCF': 2726000,
 'carenciaCF': 0,
 'mesPularParcelaCF': 0,
 'CETCF': 0.18,
 'prazoContratoCF': 120}